In [1]:
import pandas as pd
import numpy as np

pd.options.display.float_format = '{:.2f}'.format

## Carga CFI en Mercado COLOCA para Calculo Comisiones
### 1.- Query saldos de custodias:  BCS Server AI
USE BCS
GO

select cod_per Periodo,b.rut_cli Rut,b.sec_rut_cli Secuencia,b.nombre_cli Cliente,b.cod_agente,a.nemo Nemo,sum(CAN_ACTUAL + cAN_GARANTIA) AS CANTIDAD_CUOTAS
 from TBCTCIRV a 
inner join TBPLFICL b on a.rut_cli=b.rut_cli and a.sec_rut_cli=b.sec_rut_cli where cod_per='2023-06' and a.mercado='CF' and COD_BOVEDA='COR' group by cod_per,a.nemo,b.rut_cli,b.sec_rut_cli,b.nombre_cli,b.cod_agente
## 1.- ejecutar query de saldo de custodia al mes anterior
### 2.- Crear Archivo de carga,  con dos pestañas segun formato anterior
### 3.- Revisar Nemos sin Custodia 
### 4.- actualizar fecha periodo para archivo de carga


In [6]:

#archivo= 'Cierre Mensual CFI ENE23.xlsx'
archivo= 'Cierre Mensual CFI - Edicion - Postproceso.xlsx'
patrimonio= pd.read_excel(archivo, sheet_name='custodia')
Facturacion= pd.read_excel(archivo, sheet_name='Consolidado2', header=0).dropna()#, index_col=None, header=None)

#Facturacion.where(Facturacion[1]>0)
#Facturacion.query('Fact>0')
patrimonio['Nemo']=patrimonio['Nemo'].str.strip()
patrimonio.tail(10)

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS
9602,2025-03,78437170-0,0,INVERSIONES PROYECTA LTDA. ...,1000,FIPNPMAT,9217.00
9603,2025-03,78456300-6,0,ASESORIAS E INVERSIONES AIC LIMITADA ...,1000,FIPNPMAT,4615.00
9604,2025-03,78841570-2,0,INVERSIONES CARMEL LTDA ...,3314,FIPNPMAT,10357.00
9605,2025-03,78956200-8,0,COMPAÑIAS DE RENTAS F Y R LTDA. ...,3308,FIPNPMAT,10071.00
9606,2025-03,78989050-1,0,INVERSIONES SAN JUAN MARIA LIMITADA ...,1030,FIPNPMAT,2061.00
9607,2025-03,78989860-K,0,INVERSIONES B Y S LTDA. ...,7200,FIPNPMAT,10325.00
9608,2025-03,79685650-5,0,INVERSIONES LA PINTA LTDA. ...,65,FIPNPMAT,18701.00
9609,2025-03,89293500-9,0,INVERSIONES CARLOMAGNO S.A. ...,1710,FIPNPMAT,2000.00
9610,2025-03,96586750-3,0,NEVASA S.A. CORREDORES DE BOLSA ...,100,FIPNPMAT,51.00
9611,2025-03,96704770-8,0,VAYAL INVERSIONES S.A. ...,1000,FIPNPMAT,6237.00


In [7]:
set(Facturacion.emisor.to_list())



{'BTG', 'Compass', 'HMC'}

In [8]:
patrimonio= patrimonio.merge(Facturacion, left_on='Nemo', right_on='Nemo')#.to_excel('NuevaCargaFip.xlsx')

#Eliminar comisiones A fondos HMC
HMC= {'RUBIX': [9500], 
        'HMC':[7000],
        'ALTAVISTA': [7200], 'ALTAVISTA2': [7201],'B&A GESTION DE ACTIVOS': [9600]}
#Eliminar comisiones A fondos Sartor
Sartor ={'RUBIX': [9500],
       'BD INVESTMENTS': [7696],'BD INVESTMENTS2': [7697],
        'ASESORÍAS E INVERSIONES C&C SPA':[7699]}


ds=pd.DataFrame(Sartor).T
ds2= pd.DataFrame(HMC).T



In [9]:
ds

,0
RUBIX,9500
BD INVESTMENTS,7696
BD INVESTMENTS2,7697
ASESORÍAS E INVERSIONES C&C SPA,7699


In [10]:
patrimonio

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-03,5138644-2,0,ALIAGA BRAVO JORGE ALFREDO ...,9500,CFI-HMCBGB,9205.00,13188.00,HMC
1,2025-03,6617829-3,0,Serrano ALDERETE IVAN ERDMAN IV...,9500,CFI-HMCBGB,9063.00,13188.00,HMC
2,2025-03,7317272-1,0,ORTIZ DONAIRE ANGELA ...,9500,CFI-HMCBGB,2187.00,13188.00,HMC
3,2025-03,7622428-5,1,PEREZ VALDES ANDRES ...,9500,CFI-HMCBGB,2107.00,13188.00,HMC
4,2025-03,7853544-K,0,CONTRERAS TUDELA ORLANDO LEONAR...,9500,CFI-HMCBGB,4349.00,13188.00,HMC
...,...,...,...,...,...,...,...,...,...
1585,2025-03,79857030-7,0,SOCIEDAD DE INVERSIONES Y ASES.POCURO LTDA ...,3308,CFIPD7BP-E,74099.00,4692156.00,HMC
1586,2025-03,79950380-8,0,INVERSIONES RAICES LTDA. ...,1000,CFIPD7BP-E,37050.00,4692156.00,HMC
1587,2025-03,96641400-6,0,INVERSIONES IBIZA SPA ...,1000,CFIPD7BP-E,111148.00,4692156.00,HMC
1588,2025-03,96697680-2,0,INVERSIONES NOVA ROYAL LTDA. ...,1001,CFIPD7BP-E,37050.00,4692156.00,HMC


In [11]:
Nemo=Facturacion['Nemo'].to_list()
Nemo
#Facturacion['Fact'].sum()


['CFISCCH     ',
 'CFIBTGRE-E',
 'CFIBPDCCHA',
 'CFIBTGDAPA',
 'CFIBTGPLAA',
 'CFIBTGCYFA  ',
 'CFIBTGRCA   ',
 'CFICELDI1',
 'CFIBTGFGIA',
 'CFIBTGRRH       ',
 'CFICELDI2',
 'CFIBTGRRA',
 'CFIBCCHIGA',
 'CFIBTGDEA',
 'CFICGSCH-A /CFICGSCH-I',
 'CFICGGTA-E',
 'CFI-HMCBGB',
 'CFIPD7BP-E',
 'CFIHMCDPGP',
 'CFI-HMCDGD',
 'CFIHMCREPA',
 'CFI-HMCUSD / CFIHMCUSBP']

In [9]:
#i='CFITACTI-A'
#patrimonio['cuotasNVS']=patrimonio.query('Nemo=="CFITACTI-A"')['CANTIDAD_CUOTAS'].sum()
#patrimonio['prorratra']=patrimonio['CANTIDAD_CUOTAS']*202789.075630252/patrimonio['cuotasNVS']
#print(patrimonio.query('Nemo=="CFISCCH"')) # si da cero es por problema de datos, losnemos tienen espacios

#patrimonio['cuotasNVS']=patrimonio.query('Nemo=="CFITACTI-A"')['CANTIDAD_CUOTAS'].sum()
#patrimonio

In [12]:
#No paga

No_pago_Sartor =[9500,7696,7697,7699]
No_pago_HMC =  [6800, 7000, 7200, 7201, 7600, 7696, 7697, 9500, 9600]  #[9500,7000, 7200,7201,9600]
No_pago_Credicorp = [9500, 7600, 7697, 7696]
No_pago_BCI =[9500]
No_pago_Toesca=[6800]
No_pago_MBI =[9500, 7600, 7697, 7696]
No_pago_BTG =[9500]
No_pago_Compass=[9500, 7600, 7697, 7696]



test =patrimonio
for index in range(test.shape[0]):
    if test.iloc[index,8]=='HMC' and test.iloc[index,4] in No_pago_HMC:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Sartor' and test.iloc[index,4] in No_pago_Sartor:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
# nuevo 2025-03
# no pago credicorp

for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Credicorp' and test.iloc[index,4] in No_pago_Credicorp:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
#No pago BCI
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='BCI' and test.iloc[index,4] in No_pago_BCI:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0

#No pago Toesca
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Toesca' and test.iloc[index,4] in No_pago_Toesca:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
#no pago MBI
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='MBI' and test.iloc[index,4] in No_pago_MBI:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
#no pago BTG

for index in range(test.shape[0]):        
    if test.iloc[index,8]=='BTG' and test.iloc[index,4] in No_pago_BTG:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0
        
for index in range(test.shape[0]):        
    if test.iloc[index,8]=='Compass' and test.iloc[index,4] in No_pago_Compass:#
        test.iloc[index,3]= 'No Pago'
        test.iloc[index,6]=0

In [13]:
test.query('Cliente=="No Pago"')

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-03,5138644-2,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
1,2025-03,6617829-3,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
2,2025-03,7317272-1,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
3,2025-03,7622428-5,1,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
4,2025-03,7853544-K,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
...,...,...,...,...,...,...,...,...,...
1573,2025-03,76412597-5,0,No Pago,7000,CFIPD7BP-E,0.00,4692156.00,HMC
1574,2025-03,76421292-4,0,No Pago,7000,CFIPD7BP-E,0.00,4692156.00,HMC
1575,2025-03,76973031-1,0,No Pago,7000,CFIPD7BP-E,0.00,4692156.00,HMC
1578,2025-03,77980560-3,0,No Pago,7000,CFIPD7BP-E,0.00,4692156.00,HMC


In [14]:

print(index)

1589


In [15]:
test

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor
0,2025-03,5138644-2,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
1,2025-03,6617829-3,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
2,2025-03,7317272-1,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
3,2025-03,7622428-5,1,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
4,2025-03,7853544-K,0,No Pago,9500,CFI-HMCBGB,0.00,13188.00,HMC
...,...,...,...,...,...,...,...,...,...
1585,2025-03,79857030-7,0,SOCIEDAD DE INVERSIONES Y ASES.POCURO LTDA ...,3308,CFIPD7BP-E,74099.00,4692156.00,HMC
1586,2025-03,79950380-8,0,INVERSIONES RAICES LTDA. ...,1000,CFIPD7BP-E,37050.00,4692156.00,HMC
1587,2025-03,96641400-6,0,INVERSIONES IBIZA SPA ...,1000,CFIPD7BP-E,111148.00,4692156.00,HMC
1588,2025-03,96697680-2,0,INVERSIONES NOVA ROYAL LTDA. ...,1001,CFIPD7BP-E,37050.00,4692156.00,HMC


In [16]:

Nemo=Facturacion.query('Fact>0').copy()
Nemo=Nemo['Nemo'].to_list()
print(Nemo)
print(len(Nemo))


['CFISCCH     ', 'CFIBTGRE-E', 'CFIBPDCCHA', 'CFIBTGPLAA', 'CFIBTGCYFA  ', 'CFIBTGRCA   ', 'CFICELDI1', 'CFIBTGFGIA', 'CFIBTGRRH       ', 'CFICELDI2', 'CFIBTGRRA', 'CFIBCCHIGA', 'CFIBTGDEA', 'CFICGSCH-A /CFICGSCH-I', 'CFICGGTA-E', 'CFI-HMCBGB', 'CFIPD7BP-E', 'CFIHMCDPGP', 'CFI-HMCDGD', 'CFIHMCREPA', 'CFI-HMCUSD / CFIHMCUSBP']
21


In [17]:
for i in Nemo:
    if patrimonio.query('Nemo=="'+i+'"').empty :
        print(i)

CFISCCH     
CFIBTGRE-E
CFIBTGCYFA  
CFIBTGRCA   
CFIBTGRRH       
CFICGSCH-A /CFICGSCH-I
CFICGGTA-E
CFI-HMCUSD / CFIHMCUSBP


In [23]:
inic=Nemo[0]
print(inic)
Carga=patrimonio.query('Nemo=="'+inic+'"').copy()

Carga['cuotasNVS']=Carga.query('Nemo=="'+inic+'"')['CANTIDAD_CUOTAS'].sum()
Carga['monto']=Carga['CANTIDAD_CUOTAS']
Carga['acciones']=Carga['CANTIDAD_CUOTAS']
print('1')
Carga['comision']=Carga['CANTIDAD_CUOTAS']*float(Facturacion.query('Nemo=="'+inic+'"')['Fact'].iloc[0])/Carga['cuotasNVS']
Carga['porcentaje']=Carga['CANTIDAD_CUOTAS']/Carga['cuotasNVS']
print('1')
Carga['emisor'] = Facturacion.query('Nemo=="'+inic+'"').iat[0,2]
Carga['mercado'] = 'RV'
Carga['FECHA'] = '30-07-2022'

CFISCCH     
1
1


In [24]:
#Carga=patrimonio.query('Nemo=="CFISCCH"').copy()
Carga

,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor,cuotasNVS,monto,acciones,comision,porcentaje,mercado,FECHA


In [26]:
for i in Nemo[1:]:
    if patrimonio.query('Nemo=="'+i+'"').empty==False:
        dfpaso=patrimonio.query('Nemo=="'+i+'"').copy()
        dfpaso['cuotasNVS']=dfpaso.query('Nemo=="'+i+'"')['CANTIDAD_CUOTAS'].sum()
        dfpaso['monto']=dfpaso['CANTIDAD_CUOTAS']
        dfpaso['acciones']=dfpaso['CANTIDAD_CUOTAS']
        dfpaso['comision']=dfpaso['CANTIDAD_CUOTAS']*float(Facturacion.query('Nemo=="'+i+'"')['Fact'].iloc[0])/dfpaso['cuotasNVS']
        dfpaso['porcentaje']=dfpaso['CANTIDAD_CUOTAS']/dfpaso['cuotasNVS']
        dfpaso['emisor'] = Facturacion.query('Nemo=="'+i+'"').iat[0,2]
        dfpaso['mercado'] = 'RV'
        dfpaso['FECHA'] = '30-07-2022'
        Carga=pd.concat([Carga,dfpaso], sort=False)

###### Carga.keys()

In [27]:
Carga.reset_index()

,index,Periodo,Rut,Secuencia,Cliente,cod_agente,Nemo,CANTIDAD_CUOTAS,Fact,emisor,cuotasNVS,monto,acciones,comision,porcentaje,mercado,FECHA
0,363,2025-03,5199497-3,0,TAVERNE DENNISON ANABELLA EILY ...,7200,CFIBPDCCHA,32.00,108329.00,BTG,9229.00,32.00,32.00,375.61,0.00,RV,30-07-2022
1,364,2025-03,6062574-3,0,TAVERNE DENNISON FRANCISCO FELI...,7200,CFIBPDCCHA,138.00,108329.00,BTG,9229.00,138.00,138.00,1619.83,0.01,RV,30-07-2022
2,365,2025-03,6992251-1,0,COVARRUBIAS FERNANDEZ ALBERTO ...,3308,CFIBPDCCHA,1844.00,108329.00,BTG,9229.00,1844.00,1844.00,21644.67,0.20,RV,30-07-2022
3,366,2025-03,10841900-8,0,GUZMAN ROJAS FERNANDO ENRIQ...,3333,CFIBPDCCHA,521.00,108329.00,BTG,9229.00,521.00,521.00,6115.44,0.06,RV,30-07-2022
4,367,2025-03,12106743-9,0,KIM CHU MARCO ...,3333,CFIBPDCCHA,2.00,108329.00,BTG,9229.00,2.00,2.00,23.48,0.00,RV,30-07-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3171,1530,2025-03,77729834-8,0,No Pago,9500,CFIHMCREPA,0.00,4474.00,HMC,3878.00,0.00,0.00,0.00,0.00,RV,30-07-2022
3172,1531,2025-03,77813450-0,0,No Pago,7000,CFIHMCREPA,0.00,4474.00,HMC,3878.00,0.00,0.00,0.00,0.00,RV,30-07-2022
3173,1532,2025-03,77972127-2,0,No Pago,9500,CFIHMCREPA,0.00,4474.00,HMC,3878.00,0.00,0.00,0.00,0.00,RV,30-07-2022
3174,1533,2025-03,78010895-9,0,No Pago,9500,CFIHMCREPA,0.00,4474.00,HMC,3878.00,0.00,0.00,0.00,0.00,RV,30-07-2022


In [28]:
CargaCSV=Carga[['Rut','monto','acciones','porcentaje','comision','emisor' ,'Nemo','mercado','FECHA']]

In [29]:
CargaCSV.to_excel('CargafipLAST0325.xlsx')

In [30]:
CargaCSV.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3176 entries, 363 to 1534
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rut         3176 non-null   object 
 1   monto       3176 non-null   float64
 2   acciones    3176 non-null   float64
 3   porcentaje  3174 non-null   float64
 4   comision    3174 non-null   float64
 5   emisor      3176 non-null   object 
 6   Nemo        3176 non-null   object 
 7   mercado     3176 non-null   object 
 8   FECHA       3176 non-null   object 
dtypes: float64(4), object(5)
memory usage: 248.1+ KB
